# Evolving surfaces

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
from scipy.interpolate import interp1d

In [ ]:
starry.config.lazy = False
starry.config.quiet = True

## Isotropic map

In [ ]:
np.random.seed(3)
ydeg_max = 15
ydeg_tru = 10

inc_tru = 75.0
prot_tru = 3.0
alpha_tru = 0.025

tmax = 100.0
npts = 1000

tau_tru = 28.0
t0_tru = np.arange(-tau_tru, tmax, tau_tru)
ncmp_tru = len(t0_tru)

### Component amplitude as a function of time

In [ ]:
time = np.linspace(0, tmax, npts)
amp = np.zeros((ncmp_tru, npts))
for k, t0 in enumerate(t0_tru):
    amp[k] = 1 - np.abs(t0 - time + tau_tru) / tau_tru
    amp[k][amp[k] < 0] = 0
plt.plot(time, amp.T);

### Components drawn from same isotropic process

In [ ]:
power = lambda l: 5e-3 * np.exp(-((l / 5.0) ** 2)) / (2 * l + 1)
y = [[1.0] for k in range(ncmp_tru)]
for l in range(1, ydeg_max + 1):
    for k in range(ncmp_tru):
        if l > ydeg_tru:
            y[k] += list(np.zeros(2 * l + 1))
        else:
            y[k] += list(np.random.randn(2 * l + 1) * np.sqrt(power(l)))
y = np.array(y)

### Compute the flux (no differential rotation)

In [ ]:
map = starry.Map(ydeg_max)
map.inc = inc_tru
map.alpha = 0.0
flux0 = np.zeros_like(time)
for k in range(ncmp_tru):
    theta = 360.0 / prot_tru * (time - t0_tru[k])
    map[:, :] = y[k]
    flux0 += amp[k] * map.flux(theta=theta)

plt.plot(time, flux0);

In [ ]:
res = 300
downsamp = 10
nim = len(time[::downsamp])
img = np.zeros((nim, res, res))
for k in tqdm(range(ncmp_tru)):
    theta = 360.0 / prot_tru * (time - t0_tru[k])
    map[:, :] = y[k]
    img += amp[k, ::downsamp].reshape(-1, 1, 1) * map.render(
        projection="moll", res=res, theta=theta[::downsamp]
    )
map.show(image=img, projection="moll")

### Add differential rotation

In [ ]:
map = starry.Map(ydeg_max)
map.inc = inc_tru
map.alpha = alpha_tru
flux = np.zeros_like(time)
for k in range(ncmp_tru):
    theta = 360.0 / prot_tru * (time - t0_tru[k])
    map[:, :] = y[k]
    flux += amp[k] * map.flux(theta=theta)

In [ ]:
plt.plot(time, flux0, lw=1, alpha=0.5)
plt.plot(time, flux);

In [ ]:
res = 300
downsamp = 10
nim = len(time[::downsamp])
img = np.zeros((nim, res, res))
for k in tqdm(range(ncmp_tru)):
    theta = 360.0 / prot_tru * (time - t0_tru[k])
    map[:, :] = y[k]
    img += amp[k, ::downsamp].reshape(-1, 1, 1) * map.render(
        projection="moll", res=res, theta=theta[::downsamp]
    )
map.show(image=img, projection="moll")

## Discrete spots

In [ ]:
np.random.seed(0)
ydeg_max = 20
ydeg_tru = 15

inc_tru = 75.0
prot_tru = 3.0
alpha_tru = 0.025

tmax = 100.0
npts = 1000

tau_tru = 28.0
t0_tru = np.arange(-tau_tru, tmax, tau_tru)
ncmp_tru = len(t0_tru)

### Component amplitude as a function of time

In [ ]:
time = np.linspace(0, tmax, npts)
amp = np.zeros((ncmp_tru, npts))
for k, t0 in enumerate(t0_tru):
    amp[k] = 1 - np.abs(t0 - time + tau_tru) / tau_tru
    amp[k][amp[k] < 0] = 0
plt.plot(time, amp.T);

### Spot components

In [ ]:
map = starry.Map(ydeg_max)
y = np.zeros((ncmp_tru, (ydeg_max + 1) ** 2))
for k in range(ncmp_tru):
    map.reset()
    for j in range(4):
        lat = (np.arccos(2 * np.random.random() - 1) - 0.5 * np.pi) * 180 / np.pi
        lon = 360.0 * np.random.random()
        sigma = 0.01 + 0.1 * np.random.random()
        map.add_spot(lat=lat, lon=lon, sigma=sigma, intensity=-0.1, relative=False)
    map[ydeg_tru + 1 :, :] = 0.0
    y[k] = map.y

### Compute the flux (no differential rotation)

In [ ]:
map = starry.Map(ydeg_max)
map.inc = inc_tru
map.alpha = 0.0
flux0 = np.zeros_like(time)
for k in range(ncmp_tru):
    theta = 360.0 / prot_tru * (time - t0_tru[k])
    map[:, :] = y[k]
    flux0 += amp[k] * map.flux(theta=theta)

plt.plot(time, flux0);

In [ ]:
res = 300
downsamp = 10
nim = len(time[::downsamp])
img = np.zeros((nim, res, res))
for k in tqdm(range(ncmp_tru)):
    theta = 360.0 / prot_tru * (time - t0_tru[k])
    map[:, :] = y[k]
    img += amp[k, ::downsamp].reshape(-1, 1, 1) * map.render(
        projection="moll", res=res, theta=theta[::downsamp]
    )
map.show(image=img * np.pi, projection="moll", colorbar=True)

### Add differential rotation

In [ ]:
map = starry.Map(ydeg_max)
map.inc = inc_tru
map.alpha = alpha_tru
flux = np.zeros_like(time)
for k in range(ncmp_tru):
    theta = 360.0 / prot_tru * (time - t0_tru[k])
    map[:, :] = y[k]
    flux += amp[k] * map.flux(theta=theta)

In [ ]:
plt.plot(time, flux0, lw=1, alpha=0.5)
plt.plot(time, flux);

In [ ]:
res = 300
downsamp = 10
nim = len(time[::downsamp])
img = np.zeros((nim, res, res))
for k in tqdm(range(ncmp_tru)):
    theta = 360.0 / prot_tru * (time - t0_tru[k])
    map[:, :] = y[k]
    img += amp[k, ::downsamp].reshape(-1, 1, 1) * map.render(
        projection="moll", res=res, theta=theta[::downsamp]
    )
map.show(image=img * np.pi, projection="moll", colorbar=True)